In [8]:
import cv2
import numpy as np


cap = cv2.VideoCapture(-1)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))

while True:
    ret, frame = cap.read()
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        cv2.imshow("frame", hsv)
        
        lower_red = np.array([0, 20, 0])
        upper_red = np.array([20, 200, 200])
        
        mask = cv2.inRange(hsv, lower_red, upper_red)
        res = cv2.bitwise_and(frame, frame, mask=mask)
        
        kernel = np.ones((5, 5), np.uint8)
        erosion = cv2.erode(mask, kernel, iterations=1)
        dilation = cv2.dilate(mask, kernel, iterations=1)
        
        # opening aim to remove false positives (noise in the background)
        # closing aim to remove false negatives (missing pixel in the foreground)
        
        opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        
        
        cv2.imshow("mask", mask)
        cv2.imshow("res", res)
        # cv2.imshow("erosion", erosion)
        # cv2.imshow("dilation", dilation)
        cv2.imshow("opening", opening)
        cv2.imshow("closing", closing)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    else:
        print(ret)
        break
    
cap.release()
cv2.destroyAllWindows()

[ WARN:0@449.878] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [218]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img2 = cv2.imread("wu3.jpg")

dark_vest = (0, 0, 0)
light_vest = (130, 200, 165)

lab_img = cv2.cvtColor(img2, cv2.COLOR_BGR2LAB)
mask_vest = cv2.inRange(lab_img, dark_vest, light_vest)

hsv_img = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
dark_skin = (4, 44, 165)
light_skin = (15, 117, 236)
hsv_blur =  cv2.bilateralFilter(hsv_img, 15, 75, 75)
mask_skin = cv2.inRange(hsv_blur, dark_skin, light_skin)

kernel = np.ones((5, 5), np.uint8)

sum = mask_vest + mask_skin
closing = cv2.morphologyEx(sum, cv2.MORPH_CLOSE, kernel)
opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
opening2 = cv2.morphologyEx(sum, cv2.MORPH_OPEN, kernel)
erosion = cv2.erode(sum, kernel, iterations=1)
dilation = cv2.dilate(sum, kernel, iterations=1)
res = cv2.bitwise_and(img2, img2, mask=opening)
rows, cols, channels = res.shape
indent = 180
img1 = cv2.imread("yeah3.jpg")

roi = img1[-rows:, indent:indent+cols]
mask_inv = cv2.bitwise_not(opening)
img1_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

dst = cv2.add(img1_bg, res)

img1[-rows:, indent:indent+cols] = dst
edges = cv2.Canny(img2, 100, 200)
# cv2.imshow("mask", mask_skin)
# cv2.imshow("vest", mask_vest)
cv2.imshow("sum", hsv_img)
cv2.imshow("opening", opening)
cv2.imshow("res", res)
# cv2.imshow("closing", closing)
# cv2.imshow("erosion", erosion)
# cv2.imshow("dilation", dilation)
cv2.imshow("edge", edges)
# cv2.imshow("hsv", hsv_img)
# cv2.imshow("image", res)
# cv2.imshow("sum", sum)
cv2.imshow("img1", img1)

#cv2.imshow("mask", mask_inv)
# waitKey(0) will display the window infinitely until any keypress 
# (it is suitable for image display).
cv2.waitKey(0)
cv2.destroyAllWindows()

In [259]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("wu3.jpg")
img1 = cv2.imread("yeah3.jpg")
mask = np.zeros(img.shape[:2], np.uint8)

bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)

rect = (5, 45, 335, 826)
cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype("uint8")
img2 = img*mask2[:, :, np.newaxis]

_, mask3 = cv2.threshold(mask2*255, 180, 255, cv2.THRESH_BINARY)

roi = img1[-rows:, indent:indent+cols]
mask_inv = cv2.bitwise_not(mask3)

img1_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
img2_fg = cv2.bitwise_and(img2, img2, mask=mask3)


# Display the result

dst = cv2.add(img1_bg, img2_fg)

img1[-rows:, indent:indent+cols] = dst
cv2.imshow("bg", img1_bg)
cv2.imshow("fg", img2_fg)
cv2.imshow("image", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [264]:
import cv2
import numpy as np

# Create a VideoCapture object to read the video
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform thresholding to obtain the binary mask for segmentation
    _, segmented_mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)

    # Show the segmented mask
    cv2.imshow('Segmented Mask', segmented_mask)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and close the display window
cap.release()
cv2.destroyAllWindows()


[ WARN:0@18945.391] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [1]:
import cv2
import numpy as np
from mrcnn import utils
from mrcnn.model import MaskRCNN

# Path to the Mask R-CNN model weights file (downloaded from the link above)
MODEL_WEIGHTS_PATH = 'mask_rcnn_coco.h5'

# Load the pre-trained Mask R-CNN model
model = MaskRCNN(mode='inference', model_dir='./', config=utils.Config())
model.load_weights(MODEL_WEIGHTS_PATH, by_name=True)

# Open the video capture (use 0 for webcam or provide the path to a video file)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for better performance (optional)
    frame = cv2.resize(frame, None, fx=0.7, fy=0.7)

    # Perform object detection and segmentation using Mask R-CNN
    results = model.detect([frame], verbose=0)
    r = results[0]

    # Visualize the segmented regions with masks (only show the hand)
    hand_mask = r['masks'][:, :, r['class_ids'] == 1]
    hand_frame = np.where(hand_mask[:, :, 0], frame, [0, 0, 255])

    # Display the original and segmented frames
    cv2.imshow('Original Frame', frame)
    cv2.imshow('Segmented Hand', hand_frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the display window
cap.release()
cv2.destroyAllWindows()

2023-07-19 16:01:40.646933: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 16:01:41.200257: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using TensorFlow backend.


ImportError: cannot import name 'Iterable' from 'collections' (/root/miniconda3/envs/py310/lib/python3.10/collections/__init__.py)

In [297]:
import cv2
import numpy as np

def segment_hand(frame):
    # Convert the frame to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds of the skin color in HSV
    lower_bound = np.array([0, 50, 20], dtype=np.uint8)
    upper_bound = np.array([20, 255, 250], dtype=np.uint8)

    # Create a binary mask by thresholding the HSV image based on skin color
    skin_mask = cv2.inRange(hsv, lower_bound, upper_bound)

    # Apply morphological operations to remove noise and smooth the mask
    kernel = np.ones((3, 3), np.uint8)
    skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_OPEN, kernel)
    skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_CLOSE, kernel)

    # Create a masked image by applying the skin mask to the original frame
    masked_frame = cv2.bitwise_and(frame, frame, mask=skin_mask)

    return hsv, skin_mask, masked_frame

# Open the video capture (use 0 for webcam or provide the path to a video file)
cap = cv2.VideoCapture("/home/rrn4hc/mnt/Deep Learning/archive/videoplayback.mp4")
#cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for better performance (optional)
    frame = cv2.resize(frame, None, fx=0.7, fy=0.7)

    # Segment the hand from the frame
    hsv, segmented_mask, segmented_frame = segment_hand(frame)

    # Display the original and segmented frames
    cv2.imshow('Original Frame', frame)
    cv2.imshow('Segmented Mask', segmented_mask)
    cv2.imshow('Segmented Hand', segmented_frame)
    

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
    elif cv2.waitKey(10) & 0xFF == ord('p'):
        cv2.waitKey(-1)

# Release the video capture and close the display window
cap.release()
cv2.destroyAllWindows()